In [ ]:
import datetime
import math

import numpy
import pandas
from common import calc_mid_weekly_average

In [ ]:
# Earliest date that there is sufficient data for all states, including MA
earliest_date = pandas.Period('2020-03-10', freq='D')

# Set a latest date when the most recent days have garbage (like on or after holidays)
latest_date = None
if not latest_date:
    latest_date = pandas.Period((datetime.datetime.now() - datetime.timedelta(hours=19)).date(), freq='D')

In [ ]:
raw_us_stats = pandas.read_csv('https://covidtracking.com/api/v1/us/daily.csv', low_memory=False)

In [ ]:
# Remove territories
us_stats = raw_us_stats.copy()
us_stats.date = [pandas.Period(str(v)) for v in us_stats.date]

# Choose and rename a subset of columns
us_stats = us_stats[['date', 'hospitalizedCurrently', 'inIcuCurrently']]
us_stats.columns = ['Date', 'HospitalRaw', 'ICURaw']

us_stats = us_stats[us_stats.Date >= earliest_date]
us_stats = us_stats[us_stats.Date <= latest_date]
us_stats = us_stats.set_index(['Date']).sort_index()

us_stats['Hospital'], us_stats['Hospital7'] = calc_mid_weekly_average(us_stats.HospitalRaw.cumsum())
us_stats['ICU'], us_stats['ICU7'] = calc_mid_weekly_average(us_stats.ICURaw.cumsum())

In [ ]:
us_stats.tail(20)

In [ ]:
foo = us_stats[['Hospital7', 'ICU7']].iloc[180:-2].copy()
foo.columns = ['Hospitalizations', 'ICU Beds']
fam = foo.plot(
    title="Current Hospitalizations and ICU Beds", secondary_y='ICU Beds', figsize=(15,5),
)